In [30]:
import numpy as np
import pandas as pd
import scipy
from gym import error, spaces, utils
from collections import OrderedDict
from joblib import dump, load

Due to my dimensions space is 330 I'm going to take only a part of them, specifically, the most significant permissions both benign and malicious

In [100]:
np.random.seed(2)
N_STATES = 8 # the length of the dimensional world, the number of permissions
ACTIONS = scipy.math.factorial(N_STATES) # available actions, 13 combination of permissions 
EPSILON = 0.9 # greedy factor
ALPHA = 0.1  # learning rate
GAMMA = 0.9 # discount factor
MAX_EPISODES = 13 # maximum episodes
FRESH_TIME = 0.3 # fresh time for one move 
permissions_names= ['android.permission.INTERNET',                  
'android.permission.READ_PHONE_STATE',          
'android.permission.ACCESS_NETWORK_STATE',      
'android.permission.WRITE_EXTERNAL_STORAGE',    
'android.permission.ACCESS_WIFI_STATE',         
'android.permission.READ_SMS',                  
'android.permission.WRITE_SMS',                 
'android.permission.RECEIVE_BOOT_COMPLETED']
complete_permissions = pd.read_csv('../../datasets/android_permissions.csv', header=0, sep=';')

In [101]:
complete_permissions.drop(complete_permissions.index, inplace=True)
complete_permissions = pd.DataFrame(np.zeros((1,len(list(complete_permissions.columns.values)))), columns=list(complete_permissions.columns.values)).append(complete_permissions, ignore_index=True).copy()
complete_permissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 331 entries, android to type
dtypes: float64(331)
memory usage: 2.7 KB


In [102]:
complete_permissions = complete_permissions.drop('type', axis=1).copy()
complete_permissions = complete_permissions.drop('android', axis=1).copy()

In [103]:
complete_permissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 329 entries, android.app.cts.permission.TEST_GRANTED to test_permission
dtypes: float64(329)
memory usage: 2.6 KB


In [25]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
    np.zeros((actions, n_states)), # q_table initial values
    columns = permissions_names # permissions
    )
    return table

build_q_table(N_STATES, ACTIONS).head()

,android.permission.INTERNET,android.permission.READ_PHONE_STATE,android.permission.ACCESS_NETWORK_STATE,android.permission.WRITE_EXTERNAL_STORAGE,android.permission.ACCESS_WIFI_STATE,android.permission.READ_SMS,android.permission.WRITE_SMS,android.permission.RECEIVE_BOOT_COMPLETED
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(permissions_names)
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name

choose_action(1, build_q_table(N_STATES, ACTIONS))

'android.permission.INTERNET'

Let's create the environment which interacts with the defense model. First, I'm going to load the model previously trained in <a href="data analysis - Android.ipynb">Android Notebook </a>

In [31]:
clf = load('defense.joblib') 

In [104]:
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    pred = clf.predict(S)
    if pred == 0:    # it is detected as benign 
    #    if S == N_STATES - 2:   # terminate
    #        S_ = 'terminal'
    #        R = 1
    #    else:
    #        S_ = S + 1
    #        R = 0
    #else:   # move left
    #    R = 0
    #    if S == 0:
    #        S_ = S  # reach the wall
    #    else:
    #        S_ = S - 1
    #return S_, R
get_env_feedback(complete_permissions, 0)

[0]
